In [1]:
import torch
import torchvision
import numpy as np
import torchvision.transforms as tvtransforms
from torch.utils.data import DataLoader
import torch.nn as nn
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
initial_lr = 1.0
lr_decay = 0.998
initial_momentum = 0.5
final_momentum = 0.99
momentum_epochs = 15
max_norm = 15.0 
batch_size = 64
epochs = 20
dropout_hidden = 0.5
dropout_input = 0.2
weight_std = 0.01
# paper trains models for 3000 epochs . Init Lr is 10 . Init decay is 0.998

In [4]:
raw_transform = tvtransforms.ToTensor()
raw_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=raw_transform)

# Compute mean and std
loader = DataLoader(raw_dataset, batch_size=60000, shuffle=False)
data_iter = iter(loader)
images, _ = next(data_iter)
mean = images.mean().item()
std = images.std().item()

print(f"Calculated mean: {mean:.4f}, std: {std:.4f}")

Calculated mean: 0.1307, std: 0.3081


In [5]:
mnist_train_transform = tvtransforms.Compose([
    tvtransforms.RandomRotation(10),
    tvtransforms.RandomAffine(0, translate=(0.1, 0.1)),
    tvtransforms.ToTensor(), 
    tvtransforms.Normalize((mean,), (std,)),
])

mnist_test_transform = tvtransforms.Compose([
    tvtransforms.ToTensor(), 
    tvtransforms.Normalize((mean,), (std,)),
])

In [6]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=mnist_train_transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=mnist_test_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
def constrain_weights(module):
    for name, param in module.named_parameters():
        if 'weight' in name:
            l2_norms = torch.sqrt(torch.sum(param**2, dim=1, keepdim=True))
            scale = torch.clamp(torch.sqrt(torch.tensor(max_norm)) / (l2_norms + 1e-12), max=1.0)
            param.data *= scale

In [8]:
class NN(nn.Module):
    def __init__(self, layer_sizes):
        super(NN, self).__init__()
        layers = []
        layers.append(nn.Dropout(p=dropout_input))
        for i in range(len(layer_sizes) - 2):
            layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(p=dropout_hidden))
        layers.append(nn.Linear(layer_sizes[-2], layer_sizes[-1]))
        self.network = nn.Sequential(*layers)
        
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=weight_std)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        x = x.view(-1, 784)
        return self.network(x)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=initial_lr, momentum=initial_momentum)
optimizer.param_groups

In [17]:
def train_model(model, train_loader, epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=initial_lr, momentum=initial_momentum)
    train_acc = []
    
    for epoch in range(epochs):
        total=0
        correct=0
        if epoch < momentum_epochs:
            momentum = initial_momentum + (final_momentum - initial_momentum) * epoch / momentum_epochs
        else:
            momentum = final_momentum
        optimizer.param_groups[0]['momentum'] = momentum
        
        lr = initial_lr * (lr_decay ** epoch) * (1 - momentum)
        optimizer.param_groups[0]['lr'] = lr
        
        model.train()
        for data, target in tqdm(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            constrain_weights(model)
        train_acc.append(correct/total)
            
    
    return model,train_acc

In [18]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    errors = total - correct
    error_rate = (errors / total) * 100
    return errors, error_rate

In [19]:
fnn_configs = [
    [784, 800, 800, 10],
    [784, 1200, 1200, 10],
    [784, 1200, 1200, 1200, 10]
]

In [22]:
ffnn_results = {}
for config in fnn_configs:
    print(f"Training FNN with architecture: {config}")
    model = NN(config).to(device)
    model,acc  = train_model(model, train_loader, epochs)
    errors, error_rate = evaluate_model(model, test_loader)
    ffnn_results[str(config)] = [acc,errors,error_rate]
    print(f"FNN {config}: {errors} errors ({error_rate:.2f}%)")

Training FNN with architecture: [784, 800, 800, 10]


100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [00:02<00:00, 68.55it/s]


FNN [784, 800, 800, 10]: 5276 errors (52.76%)
Training FNN with architecture: [784, 1200, 1200, 10]


100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [00:05<00:00, 29.36it/s]


FNN [784, 1200, 1200, 10]: 8991 errors (89.91%)
Training FNN with architecture: [784, 1200, 1200, 1200, 10]


100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [00:03<00:00, 42.60it/s]

FNN [784, 1200, 1200, 1200, 10]: 7890 errors (78.90%)
